# A simple introduction to TensorFlow and computation graph
Author: Jiajia Sun at University of Houston, 04/04/2019

In [1]:
import tensorflow

In [2]:
print(tensorflow.__version__)  # remember to use two dash signs before and after the word 'versionse'

1.1.0


In [3]:
import tensorflow as tf

## 1. create a computation graph

In [4]:
x = tf.Variable(3)
y = tf.Variable(4)
f = x*x*y + y + 2

In [5]:
print(x)

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>


In [6]:
print(f)

Tensor("add_1:0", shape=(), dtype=int32)


Note that, no compuationa has happened yet. The above code only creates a computation graph. The actual compuation will not happen until you execute the graph. <br>
<br>
Even the variables are not initialized yet.

## 2. execute the graph

To execute a graph, we need to open a TensorFlow session and use it to intialize variables and evaluate f (by performing the actual computations defined in the previous graph). <br>
<br>
A TensorFlow session places the operations onto devices such as CPUs and GPUs, and run them. It also holds all the variable values.

Now, let us open a session.

In [7]:
sess = tf.Session()

The next thing to do is initialize the variables, x, and y (i.e., to allocate memory and set initial values for x and y).

In [8]:
sess.run(x.initializer)

In [9]:
sess.run(y.initializer)

Now that we have initialized all the variables, we can now go ahead to evaluate f.

In [10]:
sess.run(f)

42

In [11]:
sess.close()

## 3. Simplify the execution code a little bit

In [12]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

If you use with tf.Session() as sess, the session will be automatically closed.

In [13]:
result

42

## 4. Simplify it even further

We can use global_variables_initializer() function to initialize all the variables, instead of manually running the initializer for every single variable. Note that  it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variables when it is run.

In [14]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [15]:
result

42

Or, 

In [16]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    result = sess.run(f)

In [17]:
result

42

## 5. Placeholder

You can think of a placeholder as a empty variable that will be filled with data later on. Placeholders are frequently used in TensorFlow.

In [7]:
x = tf.placeholder(tf.float32, shape=(5,10),name='x')
w = tf.placeholder(tf.float32, shape=(10,1),name='y')
b = tf.constant([[-1],[-1],[-1],[-1],[-1]], tf.float32, shape=(5,1),name='b')
#xw = tf.matmul(x,w)
#xwb = xw + b
s = tf.reduce_max(tf.matmul(x,w)+b) # reduce_max return the maximum of elements across dimentions of a tensor

In [8]:
import numpy as np
np.random.seed(42)
x_data = np.random.randn(5,10)
w_data = np.random.randn(10,1)
with tf.Session() as sess:
    #output = sess.run(s,feed_dict={x:x_data,w:w_data})
    output = s.eval(feed_dict={x:x_data, w:w_data})

In [9]:
print(output)

-1.446985


In [10]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [11]:
show_graph(tf.get_default_graph()) # graph not display properly in IE. Use Chrome instead.